In [1]:
import os
import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


import sys
sys.path.append('..')

from src.etl import RecordETL
from src.preprocess import MLPDataPipeline, Conv1DDataPipeline
from src.models import BaseMLP,BaseConv1D
from src.train import Experiment
from src.utils import get_dataset_length

In [2]:
DATA_DIR = '../data/processed_data'
SAVE_DIR = '../model_weights/dev_test'

In [3]:
etl =  RecordETL(DATA_DIR)
raw_dataset = etl.read_from_tf_records_from_local().take(8)
test_raw_dataset = etl.read_from_tf_records_from_local().take(10).skip(8)

In [9]:
pipeline = Conv1DDataPipeline(num_seconds=10)
model = BaseConv1D()
optimizer = tf.keras.optimizers.Adam()
#need to validate on different subjects
experiment_one = Experiment(data=raw_dataset,
                            pipeline=pipeline,
                            model=model,
                            optimizer=optimizer,
                            training_params={'num_epochs':10,'batch_size':8,'validation_split':.2})




In [10]:
experiment_one.run()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/10
359/359 [==============================] - 32s 90ms/step - loss: 0.6983 - Accuracy: 0.5139 - Precision: 0.5131 - Recall: 0.5834 - val_loss: 0.6917 - val_Accuracy: 0.5209 - val_Precision: 0.5043 - val_Recall: 0.50720.6983 - Accu - 15s 78ms/step - loss: 0.7017 - Accuracy: 0.5092 - Precis - 17s  - 17s 52ms/step - loss: 0.6984
Epoch 2/10
359/359 [==============================] - 26s 72ms/step - loss: 0.6905 - Accuracy: 0.5397 - Precision: 0.5349 - Recall: 0.6333 - val_loss: 0.6928 - val_Accuracy

In [11]:
experiment_one.save(SAVE_DIR)

In [12]:
dataset = pipeline.fit_transform(raw_dataset)
test_dataset = pipeline.fit_transform(test_raw_dataset)

In [ ]:
get_dataset_length(dataset), get_dataset_length(test_dataset)

In [ ]:
X = []
y_true = []
for record in test_dataset:
    X.append(record[0])
    y_true.append(record[1])
X = tf.stack(X)
y_true = tf.stack(y_true).numpy().flatten()
y_pred = np.round(experiment_one.model.predict(X).flatten())
(y_pred == y_true).mean()

In [13]:
X = []
y_true = []
for record in dataset:
    X.append(record[0])
    y_true.append(record[1])
X = tf.stack(X)
y_true = tf.stack(y_true).numpy().flatten()
y_pred = np.round(experiment_one.model.predict(X).flatten())
(y_pred == y_true).mean()

0.600613154960981

In [14]:
model = BaseConv1D()
X = []
y_true = []
for record in dataset:
    X.append(record[0])
    y_true.append(record[1])
X = tf.stack(X)
y_true = tf.stack(y_true).numpy().flatten()
y_pred = np.round(model.predict(X).flatten())
(y_pred == y_true).mean()

0.5209030100334449

In [17]:
model = BaseConv1D()
model.load_weights(os.path.join(SAVE_DIR,'2020032101391584754764'))
X = []
y_true = []
for record in dataset:
    X.append(record[0])
    y_true.append(record[1])
X = tf.stack(X)
y_true = tf.stack(y_true).numpy().flatten()
y_pred = np.round(model.predict(X).flatten())
(y_pred == y_true).mean()

0.600613154960981

In [ ]:
import yaml

In [ ]:
with open('../configs/test_config.yaml','rb') as f:
    a = yaml.load(f, Loader=yaml.BaseLoader)

In [30]:
import sys
sys.path.append('..')
from src.utils import load_config_file
%autoreload

In [31]:
config = load_config_file('../configs/test_config.yaml')
config

{'data': {'train_dir': 'data/processed_data'},
 'pipeline': {'name': 'Conv1DDataPipeline', 'params': {'num_seconds': 10}},
 'model': {'name': 'BaseConv1D', 'params': {}},
 'training': {'training_loop': {'params': {'num_epochs': 50,
    'batch_size': 8,
    'validation_split': '.2'}},
  'optimizer': {'name': 'Adam', 'params': {'learning_rate': 0.0001}}},
 'float_params': '-learning_rate -validaiton_split',
 'int_params': '-batch_size -num_epochs -num_seconds'}

In [4]:
config

{'model': {'name': 'BaseConv1D', 'params': ''},
 'training': {'training_params': {'learning_rate': '.0001',
   'num_epochs': 50,
   'batch_size': 8,
   'validation_split': '.2'}}}

In [ ]:
pass

#TODO Next
Remove notebooks/specify in README these are just for testing

set up proper ETL
-modify create_tf_records to pull just abc dataset
-ensure edf is on proper channel
-allocate abc to train and val


design experiments